In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import torch
import torch.nn as nn
import torch.optim as optim
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#setup and importing
input_file='./weather&fire_2020-2022.csv'
data=pd.read_csv(input_file,error_bad_lines=False)
data.head()
print(data.columns)

<ipython-input-3-5a7c359166e8>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data=pd.read_csv(input_file,error_bad_lines=False)


FileNotFoundError: ignored

In [ ]:
data.head()
print(data.dtypes)


In [ ]:
data.drop_duplicates(inplace=True)
data.dropna(axis=0,inplace=True)

# Create additional columns

unique_values = data['Fire'].unique()  # Replace 'column_name' with the name of your column
print(unique_values)
countf=data['Fire'].value_counts()[unique_values[0]]
countt = data['Fire'].value_counts()[unique_values[1]]
print(f"The count of 'true' in 'fire' is: {countt}")
print(f"The count of 'false' in 'fire' is: {countf}")

data = data.drop(data[data['Fire'] == unique_values[0]].sample(frac=0.98).index)
# Encode the strength labels
print(data.columns)
#head.data()

In [ ]:
unique_values = data['Fire'].unique()  # Replace 'column_name' with the name of your column
print(unique_values)
countf=data['Fire'].value_counts()[unique_values[0]]
countt = data['Fire'].value_counts()[unique_values[1]]
print(f"The count of 'true' in 'fire' is: {countt}")
print(f"The count of 'false' in 'fire' is: {countf}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Split data into training and testing sets
X = data.drop(['Unnamed: 0','LAT','LON','year','month','day','date','Fire'], axis=1)
print(X.columns)
X.head()
scaler = MinMaxScaler()
columnstoscale=['ALLSKY_SFC_SW_DWN',
       'ALLSKY_SFC_LW_DWN', 'ALLSKY_SFC_UV_INDEX', 'WS2M', 'T2M', 'TS',
       'T2M_MAX', 'T2M_MIN', 'QV2M', 'PRECTOTCORR', 'PS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WD10M', 'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'WD50M'
       ]
X[columnstoscale] = scaler.fit_transform(X[columnstoscale])
X.head()
y = data['Fire']

X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Xtrain")
X_train.head()
print(X_train.shape[1])

In [ ]:
print("Ytrain")
y_train.head()
y_train.tail()


In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Assuming X_train, X_test, y_train, y_test are your numpy arrays

# Convert your data into PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).view(-1, 1).float()  # Make sure the y_train tensor has the correct dimensions
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).view(-1, 1).float()  # Make sure the y_test tensor has the correct dimensions

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.dropout = nn.Dropout(p=0.1)  # Add dropout with a probability of 0.2
        self.fc2 = nn.Linear(64, 32)
        self.out = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout to the hidden layer
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.out(x))
        return x

# Initialize the model
input_size = X_train.shape[1]
model = NeuralNetwork(input_size)

# Define the loss and the optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can choose different optimizers and learning rates

# Training loop
epochs = 1000
train_losses = []
val_losses = []
for epoch in range(epochs):
    optimizer.zero_grad()  # Clear the gradients
    outputs = model(X_train_tensor)  # Forward pass
    loss = criterion(outputs, y_train_tensor)  # Compute the loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

    # Append the training and validation loss for each epoch
    train_losses.append(loss.item())
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_tensor)
        val_losses.append(val_loss.item())
plt.figure(figsize=(8, 6))
plt.plot(range(1, epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss Over Epochs')
plt.show()
# Evaluate the model on the test set
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predicted = (test_outputs > 0.5).float()
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test)
    y_test_predicted = predicted.numpy().flatten()
    y_test_numpy = y_test_tensor.numpy().flatten()

print(f'Test accuracy: {accuracy * 100:.2f}%')
conf_matrix = confusion_matrix(y_test_numpy, y_test_predicted)
print("Confusion Matrix:")
print(conf_matrix)
